In [1]:
import numpy as np
from matplotlib import pyplot as plt
import time

In [2]:
n_features = 16000
n_samples = 15000
# n_features = 3
# n_samples = 10
n_classes = 2
X = np.random.randn(n_features, n_samples)
y = np.random.randint(0, n_classes, n_samples)
y = np.array([1 if i == 1 else -1 for i in y])
weights = np.random.randn(n_samples)


In [3]:
class WeakClassifier:
    def __init__(self, feature_index, feature_val, threshold, polarity, error):
        self.feature_index = feature_index
        self.feature_val = feature_val
        self.threshold = threshold
        self.polarity = polarity
        self.error = error
    
    # make a function for easier access as numpy array, example: np.array(wc)
    def __array__(self):
        return np.array([self.feature_index, self.feature_val, self.threshold, self.polarity, self.error])
    
    def __str__(self):
        return np.array(self).__str__()
    

# Their method

In [5]:
s_t = time.time()

total_pos, total_neg = 0, 0
for w, label in zip(weights, y):
    if label == 1:
        total_pos += w
    else:
        total_neg += w

classifiers = []
total_features = X.shape[0]
for index, feature in enumerate(X):
    if len(classifiers) % 1000 == 0 and len(classifiers) != 0:
        print("Trained %d classifiers out of %d" % (len(classifiers), total_features))

    applied_feature = sorted(zip(weights, feature, y), key=lambda x: x[1])

    pos_seen, neg_seen = 0, 0
    pos_weights, neg_weights = 0, 0
    min_error, best_feature, best_threshold, best_polarity = float('inf'), None, None, None
    current_idx = 0
    ws = []
    last_error = 0
    pos_seen_list = []
    for w, f, label in applied_feature:
        ws.append(w)
        # min(all before current example are positive and all after are negative, all before current example are negative and all after are positive)
        # error = sum of weights of misclassified examples
        error = min(neg_weights + total_pos - pos_weights, pos_weights + total_neg - neg_weights)
        last_error = error
        # print("error : ", error)
        if error < min_error:
            min_error = error
            # best_feature = features[index]
            best_feature = (current_idx, f)
            best_threshold = f
            best_polarity = 1 if pos_seen > neg_seen else -1

        if label == 1:
            pos_seen += 1
            pos_weights += w
        else:
            neg_seen += 1
            neg_weights += w
        current_idx += 1
        pos_seen_list.append(pos_seen)

    # clf = WeakClassifier(best_feature[0], best_feature[1], best_threshold, best_polarity)
    clf = WeakClassifier(best_feature[0], best_feature[1], best_threshold, best_polarity, min_error)
    classifiers.append(clf)

print("Time taken: %f seconds" % (time.time() - s_t))

Trained 1000 classifiers out of 16000
Trained 2000 classifiers out of 16000
Trained 3000 classifiers out of 16000
Trained 4000 classifiers out of 16000
Trained 5000 classifiers out of 16000
Trained 6000 classifiers out of 16000
Trained 7000 classifiers out of 16000
Trained 8000 classifiers out of 16000
Trained 9000 classifiers out of 16000
Trained 10000 classifiers out of 16000
Trained 11000 classifiers out of 16000
Trained 12000 classifiers out of 16000
Trained 13000 classifiers out of 16000
Trained 14000 classifiers out of 16000
Trained 15000 classifiers out of 16000
Time taken: 348.600005 seconds


# Mine

<h3>
<div style='color: red;'>
Question: for the polarity:<br>
 Direction that gives minimum weights <span style='color:pink'> -> To be consistent with finding minimum error using weights</span><br> 
    or<br>
 Direction that gives less misclassified samples???? <span style='color:pink'> -> To find threshold giving better accuracy???</span></div>
</h3>

In [4]:
s_t = time.time()
classifiers2 = []
total_features2 = X.shape[0]
# TODO parallize this
# TODO can we get rid of this loop and make them matrices?
for index, feature in enumerate(X):
    if len(classifiers2) % 1000 == 0 and len(classifiers2) != 0:
        print("Trained %d classifiers out of %d" % (len(classifiers2), total_features2))
        
    min_error, best_feature, best_threshold, best_polarity = float('inf'), None, None, None

    # sort by feature value first, then by weight, then by label
    # TODO No need for lexsort, and argsort is ok?
    sorting_indecies = np.lexsort((y, weights, feature))
    
    # s_w: srorted weights, s_f: sorted features, s_y: sorted labels
    s_w, s_f, s_y = weights[sorting_indecies], feature[sorting_indecies], y[sorting_indecies]
    
    for left, right in [(-1, 1), (1, -1)]: #@ y: -1 or 1
        left_weights = np.concatenate(([0], np.cumsum(s_w * (s_y == left))))
        right_weights = np.flip(np.cumsum(np.flip(s_w * (s_y == right))))
        right_weights = np.concatenate((right_weights, [0]))
        idx = np.argmin(left_weights + right_weights)
        if idx >= len(s_f):
            idx = len(s_f) - 1
            
        cur_min_error = left_weights[idx] + right_weights[idx]
        if cur_min_error < min_error:
            min_error = cur_min_error
            best_feature, best_threshold = (idx, s_f[idx]), s_f[idx]
            best_polarity = left

    clf2 = WeakClassifier(best_feature[0], best_feature[1], best_threshold, best_polarity, min_error)
    classifiers2.append(clf2)

    

print("Time taken: %f seconds" % (time.time() - s_t))

Trained 1000 classifiers out of 16000
Trained 2000 classifiers out of 16000
Trained 3000 classifiers out of 16000
Trained 4000 classifiers out of 16000
Trained 5000 classifiers out of 16000
Trained 6000 classifiers out of 16000
Trained 7000 classifiers out of 16000
Trained 8000 classifiers out of 16000
Trained 9000 classifiers out of 16000
Trained 10000 classifiers out of 16000
Trained 11000 classifiers out of 16000
Trained 12000 classifiers out of 16000
Trained 13000 classifiers out of 16000
Trained 14000 classifiers out of 16000
Trained 15000 classifiers out of 16000
Time taken: 38.363074 seconds


# Even more optimized?
<h1>
    <div style='color:red'>
        Unfortunately, didn't work. The computer freezes when I try to run it.
    </div>
    <div style='color:red'>
        Maybe will need to run it on a GPU??
    </div>
</h1>


In [ ]:
s_t = time.time()
classifiers3 = []
total_features3 = X.shape[0]

# TODO parallize this
# TODO can we get rid of this loop and make them matrices?
# for index, feature in enumerate(X):
#     if len(classifiers3) % 1000 == 0 and len(classifiers3) != 0:
#         print("Trained %d classifiers out of %d" % (len(classifiers3), total_features3))
        

min_error, best_feature, best_threshold, best_polarity = np.array([float('inf')]*X.shape[0])\
    , np.zeros((X.shape[0], 2)), np.zeros(X.shape[0]), np.zeros(X.shape[0])

# sort by feature value first, then by weight, then by label
# TODO No need for lexsort, and argsort is ok? looks like tile is expensive
weights2d = np.tile(weights, (X.shape[0], 1))
y2d = np.tile(y, (X.shape[0], 1))
sorting_indecies = np.lexsort((y2d
                               ,weights2d
                               ,X))
idx0 = np.arange(X.shape[0]).reshape(-1, 1)
# s_w: srorted weights, s_f: sorted features, s_y: sorted labels
s_w, s_f, s_y = weights2d[idx0, sorting_indecies], X[idx0, sorting_indecies], y2d[idx0, sorting_indecies]

for left, right in [(-1, 1), (1, -1)]: #@ y: -1 or 1
    # left_weights = np.concatenate(([0], np.cumsum(s_w * (s_y == left), axis=1)))
    left_weights = np.c_[np.zeros((s_w.shape[0], 1))
                         , np.cumsum(s_w * (s_y == left), axis=1)]
    right_weights = np.flip(np.cumsum(np.flip(s_w * (s_y == right), axis = 1), axis=1), axis=1)
    # right_weights = np.concatenate((right_weights, [0]))
    right_weights = np.c_[right_weights, np.zeros((right_weights.shape[0], 1))]
    idx = np.argmin(left_weights + right_weights, axis=1)
    # should it be zero???
    idx[idx >= s_f.shape[1]] = s_f.shape[1] - 1
    # if idx >= len(s_f):
    #     idx = len(s_f) - 1
    ii1 = np.arange(idx.shape[0])
    cur_min_error = left_weights[ii1, idx] + right_weights[ii1, idx]
    temp_bool = cur_min_error < min_error
    min_error[temp_bool] = cur_min_error[temp_bool]
    selected_idx = idx[temp_bool]
    selected_features = s_f[ii1[temp_bool], selected_idx]
    best_feature[temp_bool] = np.array(list(zip(selected_idx, selected_features)))
    best_threshold[temp_bool] = s_f[ii1[temp_bool], idx[temp_bool]]
    best_polarity[temp_bool] = left
    # if cur_min_error < min_error:
    #     min_error = cur_min_error
    #     best_feature, best_threshold = (idx, s_f[idx]), s_f[idx]
    #     best_polarity = left
classifiers3 = [WeakClassifier(*clf3) for clf3 in zip(best_feature[:,0], best_feature[:,1], best_threshold, best_polarity, min_error)]
# clf3 = WeakClassifier(best_feature[0], best_feature[1], best_threshold, best_polarity, min_error)
# classifiers3.append(clf2)

    

print("Time taken: %f seconds" % (time.time() - s_t))

In [110]:
for el in classifiers:
    print(el)
print()
for el in classifiers2:
    print(el)
print()
for el in classifiers3:
    print(el)
# classifiers==classifiers2
# print(classifiers[0])
# print(classifiers2[0])

[ 7.          0.33365897  0.33365897  1.         -2.58596895]
[ 6.          0.83123379  0.83123379 -1.         -4.38605707]
[ 5.         -0.17706    -0.17706     1.         -1.82290006]

[ 7.          0.33365897  0.33365897 -1.         -2.58596895]
[ 6.          0.83123379  0.83123379  1.         -4.38605707]
[ 5.         -0.17706    -0.17706     1.         -1.82290006]

[ 7.          0.33365897  0.33365897 -1.         -2.58596895]
[ 6.          0.83123379  0.83123379  1.         -4.38605707]
[ 5.         -0.17706    -0.17706     1.         -1.82290006]


In [119]:
a1 = np.zeros((len(classifiers), 5))
for i in range(len(classifiers)):
    a1[i] = np.array(classifiers[i])

a2 = np.zeros((len(classifiers2), 5))
for i in range(len(classifiers2)):
    a2[i] = np.array(classifiers2[i])
a3 = np.zeros((len(classifiers3), 5))
for i in range(len(classifiers3)):
    a3[i] = np.array(classifiers3[i])
    
print(a1[a1!=a2])
print(a1)
print()
print(a2)
print()
print(a3)
print(np.argwhere(a2!=a3))
print(np.argwhere(a2!=a1))
print(a2[a1!=a2])

[ 1.         -1.         -4.38605707 -1.82290006]
[[ 7.          0.33365897  0.33365897  1.         -2.58596895]
 [ 6.          0.83123379  0.83123379 -1.         -4.38605707]
 [ 5.         -0.17706    -0.17706     1.         -1.82290006]]

[[ 7.          0.33365897  0.33365897 -1.         -2.58596895]
 [ 6.          0.83123379  0.83123379  1.         -4.38605707]
 [ 5.         -0.17706    -0.17706     1.         -1.82290006]]

[[ 7.          0.33365897  0.33365897 -1.         -2.58596895]
 [ 6.          0.83123379  0.83123379  1.         -4.38605707]
 [ 5.         -0.17706    -0.17706     1.         -1.82290006]]
[]
[[0 3]
 [1 3]
 [1 4]
 [2 4]]
[-1.          1.         -4.38605707 -1.82290006]
-4.38605706517018 -4.386057065170179 -4.386057065170179
-1.8229000640981816 -1.8229000640981814 -1.8229000640981814
